<a href="https://colab.research.google.com/github/lovepreetmultani/python/blob/main/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=8144e710d3683843b92dc5a72fa4ecacb58c049bb664e66abd764b46690b7b76
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [4]:
spark=SparkSession.builder\
 .master("local[*]")\
 .appName("WordCount")\
 .getOrCreate()
sc=spark.sparkContext

In [11]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [12]:
dataPath = "drive/My Drive/Datasets/pg4300.txt"

In [14]:
data_rdd = sc.textFile(dataPath)
data_rdd.take(10)

['The Project Gutenberg EBook of Ulysses, by James Joyce',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '',
 'Title: Ulysses',
 '']

In [ ]:
#data_rdd.collect()

In [15]:
data_rdd.count()

33055

In [17]:
def lower_clean_str(x):
 punc = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
 lowercased_str = x.lower()
 for ch in punc:
  lowercased_str = lowercased_str.replace(ch, '')
  return lowercased_str  

In [18]:
data_rdd = data_rdd.map(lower_clean_str)
data_rdd.take(10)

['the project gutenberg ebook of ulysses, by james joyce',
 '',
 'this ebook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  you may copy it, give it away or',
 're-use it under the terms of the project gutenberg license included',
 'with this ebook or online at www.gutenberg.org',
 '',
 '',
 'title: ulysses',
 '']

In [21]:
data_rdd = data_rdd.flatMap(lambda line: line.split(" "))
data_rdd.take(10)

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'ulysses,',
 'by',
 'james',
 'joyce',
 '']

In [22]:
data_rdd = data_rdd.filter(lambda x:x!= '')
data_rdd.take(10)

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'ulysses,',
 'by',
 'james',
 'joyce',
 'this']

In [23]:
data_count = data_rdd.map(lambda word:(word,1))
data_count.take(5)

[('the', 1), ('project', 1), ('gutenberg', 1), ('ebook', 1), ('of', 1)]

In [24]:
data_count_RBK = data_count.reduceByKey(lambda x,y:(x+y)).sortByKey()
data_count_RBK.take(10)

[('"come', 1),
 ('"defects,"', 1),
 ('"i', 1),
 ('"information', 1),
 ('"j"', 1),
 ('"plain', 2),
 ('"project', 5),
 ('"right', 1),
 ('"viator"', 1),
 ('#4300]', 1)]

In [25]:
data_count_RBK = data_count_RBK.map(lambda x:(x[1],x[0]))
data_count_RBK.take(10)

[(1, '"come'),
 (1, '"defects,"'),
 (1, '"i'),
 (1, '"information'),
 (1, '"j"'),
 (2, '"plain'),
 (5, '"project'),
 (1, '"right'),
 (1, '"viator"'),
 (1, '#4300]')]

In [26]:
data_count_RBK.sortByKey(False).take(10)

[(14854, 'the'),
 (8216, 'of'),
 (7052, 'and'),
 (6400, 'a'),
 (4914, 'to'),
 (4797, 'in'),
 (3622, 'he'),
 (3275, 'his'),
 (2487, 'with'),
 (2444, 'i')]

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords

In [28]:
data_count_RBK = data_count_RBK.filter(lambda x: x[1] not in stopwords)
data_count_RBK.sortByKey(False).take(10)

[(706, 'like'),
 (701, 'mr'),
 (583, 'one'),
 (480, 'said.'),
 (469, 'old'),
 (454, 'says'),
 (450, 'bloom'),
 (430, 'said'),
 (369, 'would'),
 (346, 'two')]

In [29]:
data_count_RBK_sorted = data_count_RBK.sortByKey(False)
data_count_RBK_sorted.take(10)

[(706, 'like'),
 (701, 'mr'),
 (583, 'one'),
 (480, 'said.'),
 (469, 'old'),
 (454, 'says'),
 (450, 'bloom'),
 (430, 'said'),
 (369, 'would'),
 (346, 'two')]

In [31]:
data_count_RBK_sorted.saveAsTextFile('drive/My Drive/Datasets/WordCountResults.csv')